In [12]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys 

sns.set()
sys.path.append("../src/")

from models import SimulateStatement, Model
from modelstats import ModelStats
from dataclasses import dataclass
from crop import crop_statements_until_t_by_politician, crop_statements_until_t
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
df = pd.read_csv("../data/DATA_FINAL.csv")
df.time = pd.to_datetime(df.time)
df = df.sort_values(by=['time'])

In [14]:
df.iloc[-1].time

Timestamp('2019-07-11 15:30:16')

In [15]:
tau = crop_statements_until_t_by_politician(df,'2019-07-11 15:30:16', 422)

In [20]:
# sys.path.append("../data/")

data_path = "../data/DATA_FINAL.csv"
data_deputados_path = "../data/DEPUTADOS_FINAL.csv"

Experimento = ModelStats(data_path, data_deputados_path)

lambd = 0.6
delta = 0.3
lag = 10



In [22]:
times, from_time_to_politician_opinion_list, from_politician_to_opinion_history= Experimento.get_opinions(lambd, delta, lag, datetime(2020,1,1))

19it [00:01, 12.80it/s]


In [58]:
from_time_to_politician_opinion_list[]

314

In [52]:
Experimento.get_window_size_fror_probability_estimation(20)

{422: 1,
 307: 3,
 223: 2,
 477: 1,
 346: 2,
 120: 1,
 299: 3,
 430: 10,
 470: 3,
 513: 0,
 34: 7,
 348: 8,
 395: 1,
 195: 2,
 127: 6,
 280: 7,
 490: 6,
 9: 1,
 455: 8,
 74: 2,
 26: 3,
 146: 5,
 144: 0,
 488: 3,
 377: 3,
 405: 1,
 71: 3,
 106: 4,
 459: 0,
 485: 2,
 272: 2,
 342: 7,
 450: 7,
 176: 2,
 136: 1,
 169: 1,
 381: 2,
 62: 3,
 443: 2,
 95: 0,
 441: 1,
 214: 1,
 10: 1,
 43: 8,
 382: 8,
 278: 10,
 494: 3,
 425: 2,
 310: 1,
 245: 5,
 303: 1,
 273: 3,
 414: 1,
 266: 1,
 329: 5,
 436: 16,
 375: 2,
 192: 5,
 393: 1,
 396: 4,
 135: 4,
 87: 4,
 315: 8,
 188: 4,
 48: 4,
 262: 4,
 73: 5,
 279: 1,
 211: 13,
 70: 9,
 508: 14,
 4: 1,
 372: 6,
 145: 1,
 49: 8,
 255: 13,
 338: 6,
 294: 5,
 286: 6,
 358: 6,
 190: 9,
 219: 4,
 260: 3,
 199: 0,
 412: 2,
 216: 17,
 497: 23,
 334: 2,
 464: 30,
 76: 1,
 201: 20,
 251: 10,
 301: 15,
 155: 4,
 291: 5,
 369: 34,
 7: 5,
 343: 0,
 408: 1,
 479: 22,
 69: 7,
 100: 1,
 252: 3,
 149: 5,
 238: 12,
 300: 0,
 267: 3,
 325: 0,
 406: 1,
 204: 6,
 514: 0,
 20: 5,

In [53]:
politicians_opinion_histograms = Experimento.get_score_histogram(lambd,delta,lag)

  0%|          | 0/314 [00:00<?, ?it/s]


AttributeError: 'PoliticiansOpinionInTime' object has no attribute 'politicians_opinions'

In [2]:
from datetime import datetime
from dataclasses import dataclass
from typing import List

@dataclass
class PoliticianOpinion:
    politician_name: str
    opinion_score: float

@dataclass
class PoliticiansOpinionInTime:
    politician_opinions: List[PoliticianOpinion]
    datetime: datetime

def get_politician_trajectories(opinions_in_time: List[PoliticiansOpinionInTime], politician_name: str):
    """
    Get all different trajectories of opinions for a single politician.

    Parameters:
    - opinions_in_time: List of PoliticiansOpinionInTime instances.
    - politician_name: The name of the politician.

    Returns:
    - A list of trajectories for the specified politician.
    """
    politician_trajectories = []

    # Iterate through the list of opinions_in_time
    for opinion_in_time in opinions_in_time:
        datetime_point = opinion_in_time.datetime
        # Find the politician's opinion at this datetime_point
        politician_opinion = next((opinion.opinion_score for opinion in opinion_in_time.politician_opinions
                                   if opinion.politician_name == politician_name), None)

        if politician_opinion is not None:
            # Append the datetime_point and opinion to the trajectories
            politician_trajectories.append((datetime_point, politician_opinion))

    return politician_trajectories

# Example usage:
opinions_data = [
    PoliticiansOpinionInTime([PoliticianOpinion("Politician1", 0.8), PoliticianOpinion("Politician2", 0.5)], datetime(2023, 1, 1)),
    PoliticiansOpinionInTime([PoliticianOpinion("Politician1", 0.7), PoliticianOpinion("Politician2", 0.6)], datetime(2023, 1, 2)),
    PoliticiansOpinionInTime([PoliticianOpinion("Politician1", 0.6), PoliticianOpinion("Politician2", 0.7)], datetime(2023, 1, 3)),
]

politician_name_to_check = "Politician1"
trajectories = get_politician_trajectories(opinions_data, politician_name_to_check)

print(f"Opinion trajectories for {politician_name_to_check}: {trajectories}")


Opinion trajectories for Politician1: [(datetime.datetime(2023, 1, 1, 0, 0), 0.8), (datetime.datetime(2023, 1, 2, 0, 0), 0.7), (datetime.datetime(2023, 1, 3, 0, 0), 0.6)]


In [ ]:
@dataclass
class OpinionHistogram:
    """Class for identifying a single politician opinion"""
    bins: list[int]
    opinion: list[int] 

@dataclass
class ScoreHistogram:
    """Class for identifying a single politician opinion"""
    politician_id: list[int]
    opinion: list[int]